# 3.6 EXERCISES

## 3.1

** Can all matrices $A \in \mathbb{R}^{n \times n}$ be factored $A = LU$?  Why or why not?**

No. Normal LU decompostion cannot deal with some matrices (like the one in section 3.2), because in which case a diagonal element becomes 0 while forward-substitution.  However, using permutation matrix $P$ allows factorization to $A = PLU$. 

## 3.2

**Solve the following system of equations using Gaussian elimination, writing the corresponding elimination matrix of each step:$$
\begin{pmatrix}
  2 & 4 \\
  3 & 5 \\
\end{pmatrix}
\begin{pmatrix}
  x \\
  y \\
\end{pmatrix}
 = \begin{pmatrix}
 2 \\
 4 \\
\end{pmatrix}
$$Factor the matrix on the left-hand side as a product $A = LU$.**

Using augmented matrix,

\begin{align}
  \left(\begin{array}{cc|c}
    2 & 4 & 2 \\
    3 & 5 & 4 \\
  \end{array}\right)
  &\rightarrow
  \left(\begin{array}{cc|c}
    2 & 4 & 2 \\
    0 & -1 & 1 \\
  \end{array}\right) \qquad
  \left[\left(\begin{array}{cc}
    1 & 0 \\
    -3/2 & 1 \\
  \end{array}\right)\right] \\
  &\rightarrow
  \left(\begin{array}{cc|c}
    1 & 2 &  1 \\
    0 & 1 & -1 \\
  \end{array}\right) \qquad
  \left[\left(\begin{array}{cc}
    1/2 & 0 \\
    0 & -1 \\
  \end{array}\right)\right] \\
  &\rightarrow
  \left(\begin{array}{cc|c}
    1 & 0 & 3 \\
    0 & 1 & -1 \\
  \end{array}\right) \qquad
  \left[\left(\begin{array}{cc}
    1 & -2 \\
    0 &  1 \\
  \end{array}\right)\right] \\
\end{align}

This process gives us the hint of LU decomposition:

\begin{align}
  &\left(\begin{array}{cc}
    1 & 0 \\
    -3/2 & 1 \\
  \end{array}\right)
  \left(\begin{array}{cc}
    2 & 4 \\
    3 & 5 \\
  \end{array}\right)
  =
  \left(\begin{array}{cc}
    2 & 4 \\
    0 & -1 \\
  \end{array}\right) \\
  &\iff
  \left(\begin{array}{cc}
    2 & 4 \\
    3 & 5 \\
  \end{array}\right)
  =
  \left(\begin{array}{cc}
    1 & 0 \\
    3/2 & 1 \\
  \end{array}\right)
  \left(\begin{array}{cc}
    2 & 4 \\
    0 & -1 \\
  \end{array}\right) \\
\end{align}

## 3.3

**Factor the following matrix $A$ as $A = LU$.$$
\begin{pmatrix}
  1 & 2 & 7 \\
  3 & 5 & -1 \\
  6 & 1 & 4 \\
\end{pmatrix}
$$**

In [15]:
import numpy as np

def lu_without_pivoting(A, compact=False):
    """Factors A to A = LU"""
    assert A.ndim == 2
    A = A.astype(float)
    n1, n2 = A.shape
    for p in range(n1):
        for r in range(p+1, n1):
            s = -A[r, p] / A[p, p]
            A[r, p] = -s
            A[r, p+1:] += s * A[p, p+1:]
    if compact:
        return A
    else:
        n_min = min(n1, n2)
        L = np.fromfunction(lambda i, j : A*(i > j) + np.eye(n1, n_min),
                            shape=(n1, n_min), dtype=float)
        U = np.fromfunction(lambda i, j : A*(i <= j),
                            shape=(n_min, n2), dtype=float)
    return L, U

In [17]:
A = np.array([
    [1, 2, 7],
    [3, 5, -1],
    [6, 1, 4]
])
L, U = lu_without_pivoting(A)
print("L =\n", L)
print("U =\n", U)

L =
 [[  1.   0.   0.]
 [  3.   1.   0.]
 [  6.  11.   1.]]
U =
 [[   1.    2.    7.]
 [   0.   -1.  -22.]
 [   0.    0.  204.]]


## 3.4

**Modify the code in Figure 3.1 to include partial pivoting.**

In [2]:
import numpy as np

def forward_substitution_pivoting(A, b):
    """ Cnverts a system Ax = b to an upper-triangular system Ux = y.
    
    Positional arguments:
        A -- Invertible numpy array. A.shape == (n, n)
        b -- 1-dimensional Numpy array. b.shape == (n, )
    """
    assert A.ndim == 2 and b.ndim == 1 and A.shape[0] == A.shape[1] == b.size
    
    n = A.shape[0]
    
    # U will be upper triangular at completion.
    U = A.astype(float)
    y = b.astype(float)
    
    # Iterate over current pivot row p.
    for p in range(n):
        pivot = np.argmax(np.absolute(U[p:, p])) + p
        if pivot != p:
            U[p], U[pivot] = U[pivot].copy(), U[p].copy()
        s = U[p, p]
        if s != 0:
            y[p] /= s
            U[p] /= s

            for r in range(p + 1, n):
                s = U[r, p]
                y[r] -= s * y[p]
                U[r] -= s * U[p]
    return U, y